# Imports

In [1]:
# import pandas as pd
# df = pd.read_csv('data\Data\musicFeatuers.csv')

# from sklearn.preprocessing import LabelEncoder
# label_encoder = LabelEncoder()

# # df['label'] =  label_encoder.fit_transform(df['label'])

# df = df.drop(['filename'], axis=1)



# X = df[['chroma_stft_mean','chroma_stft_var','rms_mean','rms_var','spectral_centroid_mean','spectral_centroid_var','spectral_bandwidth_mean','spectral_bandwidth_var']]

# y = df['label']

# y = df[['label']]
# X = df[df.columns.difference(['label'])]

# from sklearn.model_selection import train_test_split

# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

# X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.3, random_state=1)
# ## train a tree model
# sk_dt.fit(X_train, y_train)

# from sklearn.linear_model import LinearRegression
# # model = LinearRegression()
# # model.fit(X_train, y_train)


# from sklearn.metrics import classification_report

# predictions = sk_dt.predict(X_test)


# print(classification_report(y_test, predictions))

# y_pred = sk_dt.predict(X_test)



# from sklearn.metrics import mean_squared_error
# mse = mean_squared_error(Y_test,y_pred)
# print(mse)


In [2]:
# %pip install librosa --user

In [3]:
import xgboost
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import librosa
import numpy as np


import warnings
warnings.filterwarnings('ignore')
# ## create the classifier object leaving all params to default values except random_state
# sk_dt = DecisionTreeClassifier(random_state=1)


# Model

In [4]:
df = pd.read_csv(r'..\data\Data\features_3_sec.csv')

# df = df.drop(['filename'], axis=1)

# df = df[['chroma_stft_mean','chroma_stft_var','rms_mean','rms_var','spectral_centroid_mean','spectral_centroid_var','spectral_bandwidth_mean','spectral_bandwidth_var','label']]

df = df[['chroma_stft_mean','chroma_stft_var','rms_mean','rms_var','spectral_centroid_mean','spectral_centroid_var','spectral_bandwidth_mean','spectral_bandwidth_var','rolloff_mean','rolloff_var','zero_crossing_rate_mean','zero_crossing_rate_var','harmony_mean','harmony_var','tempo','label']]
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()


df['label'] =  label_encoder.fit_transform(df['label'])

print(label_encoder.classes_)

y = df[['label']]
X = df[df.columns.difference(['label'])]


## split both X and y using a ratio of 70% training - 30% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
print(len(X_train), len(X_test), len(y_train), len(y_test))

target_name = ['blues', 'classical', 'country', 'disco', 'hiphop' ,'jazz' ,'metal', 'pop','reggae' ,'rock']


['blues' 'classical' 'country' 'disco' 'hiphop' 'jazz' 'metal' 'pop'
 'reggae' 'rock']
6993 2997 6993 2997


In [5]:


xgb = xgboost.XGBClassifier(n_estimators=1000,enable_catergorical=True,learning_rate=0.05)
xgb.fit(X_train, y_train)

## make predictions on the test portion (predict the labels of the rows from the test portion of X)
predictionsXgb = xgb.predict(X_test)



print(classification_report(y_test, predictionsXgb, target_names=target_name))
## can also output the confusion matrix
# cm = confusion_matrix(y_test, predictions)
# print(cm)

print("Accuracy: " ,metrics.accuracy_score(y_test, predictionsXgb))

cols_when_model_builds = xgb.feature_names_in_


              precision    recall  f1-score   support

       blues       0.77      0.83      0.80       302
   classical       0.91      0.92      0.92       296
     country       0.68      0.76      0.72       312
       disco       0.78      0.78      0.78       283
      hiphop       0.85      0.79      0.82       320
        jazz       0.80      0.81      0.80       289
       metal       0.87      0.89      0.88       303
         pop       0.84      0.77      0.80       298
      reggae       0.78      0.73      0.75       310
        rock       0.65      0.61      0.63       284

    accuracy                           0.79      2997
   macro avg       0.79      0.79      0.79      2997
weighted avg       0.79      0.79      0.79      2997

Accuracy:  0.7911244577911245


In [6]:
from sklearn.ensemble import RandomForestClassifier


forest = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0)
forest.fit(X_train, y_train)
predictionsForest = forest.predict(X_test)

print(classification_report(y_test, predictionsForest, target_names=target_name))
print("Accuracy: " ,metrics.accuracy_score(y_test, predictionsForest))



              precision    recall  f1-score   support

       blues       0.66      0.75      0.70       302
   classical       0.88      0.89      0.89       296
     country       0.58      0.58      0.58       312
       disco       0.58      0.72      0.64       283
      hiphop       0.80      0.64      0.71       320
        jazz       0.70      0.72      0.71       289
       metal       0.74      0.87      0.80       303
         pop       0.80      0.71      0.75       298
      reggae       0.64      0.70      0.67       310
        rock       0.64      0.40      0.50       284

    accuracy                           0.70      2997
   macro avg       0.70      0.70      0.70      2997
weighted avg       0.70      0.70      0.70      2997

Accuracy:  0.6996996996996997


In [7]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
predictionsDTC = dtc.predict(X_test)

print(classification_report(y_test, predictionsDTC, target_names=target_name))
print("Accuracy: " ,metrics.accuracy_score(y_test, predictionsDTC))

              precision    recall  f1-score   support

       blues       0.60      0.55      0.57       302
   classical       0.84      0.87      0.86       296
     country       0.47      0.44      0.45       312
       disco       0.51      0.53      0.52       283
      hiphop       0.62      0.55      0.58       320
        jazz       0.63      0.69      0.66       289
       metal       0.72      0.72      0.72       303
         pop       0.66      0.65      0.66       298
      reggae       0.56      0.56      0.56       310
        rock       0.36      0.42      0.39       284

    accuracy                           0.60      2997
   macro avg       0.60      0.60      0.60      2997
weighted avg       0.60      0.60      0.60      2997

Accuracy:  0.5969302635969302


In [61]:
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier

scaledX = preprocessing.scale(X_train)
scaledt = preprocessing.scale(X_test)


KNC = KNeighborsClassifier(n_neighbors=1)
KNC.fit(scaledX, y_train)
predictionsKNC = KNC.predict(scaledt)

print(classification_report(y_test, predictionsKNC, target_names=target_name))
print("Accuracy: " ,metrics.accuracy_score(y_test, predictionsKNC))


              precision    recall  f1-score   support

       blues       0.71      0.75      0.73       302
   classical       0.89      0.90      0.89       296
     country       0.62      0.62      0.62       312
       disco       0.68      0.71      0.69       283
      hiphop       0.78      0.76      0.77       320
        jazz       0.74      0.73      0.74       289
       metal       0.84      0.85      0.85       303
         pop       0.85      0.76      0.80       298
      reggae       0.69      0.71      0.70       310
        rock       0.63      0.63      0.63       284

    accuracy                           0.74      2997
   macro avg       0.74      0.74      0.74      2997
weighted avg       0.74      0.74      0.74      2997

Accuracy:  0.7414080747414081


# Find Similar

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

def find_sim(data):
    placeHoldername = 'test'
    data['filename'] = placeHoldername

    df_sim = pd.read_csv(r'..\data\Data\features_30_sec.csv')

    df_sim = df_sim[['filename','chroma_stft_mean','chroma_stft_var','rms_mean','rms_var','spectral_centroid_mean','spectral_centroid_var','spectral_bandwidth_mean','spectral_bandwidth_var','rolloff_mean','rolloff_var','zero_crossing_rate_mean','zero_crossing_rate_var','harmony_mean','harmony_var','tempo','label']]


    from sklearn.preprocessing import LabelEncoder
    label_encoder = LabelEncoder()
    df_sim['label'] = df_sim['label'].astype("string")
    df_sim['label'] =  label_encoder.fit_transform(df_sim['label'])




    combined_df = pd.concat([df_sim, data], ignore_index=True)

    combined_df = combined_df.set_index('filename')

 
    labels = combined_df[['label']]

    
    scaled = preprocessing.scale(combined_df)
    similarity = cosine_similarity(scaled)
    sim_df_labels = pd.DataFrame(similarity)
    sim_df_names = sim_df_labels.set_index(labels.index)
    sim_df_names.columns = labels.index

    series = sim_df_names[placeHoldername].sort_values(ascending=False)
    series = series.drop(placeHoldername)
    return series.head(3)





# Find Prediction

In [10]:
def find_pred(data, features, predicted_feature):
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LinearRegression

    features = features.drop(['filename'], axis=1)

    X_train, X_test, y_train, y_test = train_test_split(data, data[predicted_feature], test_size=0.2)

    model = LinearRegression()
    model.fit(X_train, y_train)


    prediction = model.predict(features)

    return prediction[0]




# Feature Extractor



In [11]:
def extract_features(file):
    y, sr = librosa.load(file)

    chroma_sft_mean =  np.mean(librosa.feature.chroma_stft(y=y, sr=sr))
    chroma_sft_var =  librosa.feature.chroma_stft(y=y, sr=sr).var()

    rms_mean = librosa.feature.rms(y=y).mean()
    rms_var = librosa.feature.rms(y=y).var()

    spectral_centroid_mean = librosa.feature.spectral_centroid(y=y, sr=sr).mean()
    spectral_centroid_var = librosa.feature.spectral_centroid(y=y, sr=sr).var()

    spectral_bandwith_mean = librosa.feature.spectral_bandwidth(y=y, sr=sr).mean()
    spectral_bandwith_var = librosa.feature.spectral_bandwidth(y=y, sr=sr).var()

    rolloff_mean = librosa.feature.spectral_rolloff(y=y, sr=sr).mean()
    rolloff_var = librosa.feature.spectral_rolloff(y=y, sr=sr).var()

    zero_crossing_rate_mean = librosa.feature.zero_crossing_rate(y=y).mean()
    zero_crossing_rate_var = librosa.feature.zero_crossing_rate(y=y).var()


    harmony_mean = librosa.effects.harmonic(y).mean()
    harmony_var = librosa.effects.harmonic(y).var()

    tempo = librosa.feature.tempo(y=y, sr=sr)[0]

    

    print("chroma_sft_mean: ",chroma_sft_mean)
    print("chroma_sft_var: ",chroma_sft_var)
    print("rms_mean: ",rms_mean)
    print("rms_var: ",rms_var)
    print("spectral_centroid_mean: ",spectral_centroid_mean)
    print("spectral_centroid_var: ",spectral_centroid_var)
    print("spectral_bandwith_mean: ",spectral_bandwith_mean)
    print("spectral_bandwith_var: ",spectral_bandwith_var)
    print("rolloff_mean: ",rolloff_mean)
    print("rolloff_var: ",rolloff_var)
    print("zero_crossing_rate_mean: ",zero_crossing_rate_mean)
    print("zero_crossing_rate_var: ",zero_crossing_rate_var)
    print("harmony_mean: ",harmony_mean)
    print("harmony_var: ",harmony_var)
    print("tempo: ",tempo)

    features = pd.DataFrame({'chroma_stft_mean':[chroma_sft_mean],'chroma_stft_var':[chroma_sft_var],'rms_mean':[rms_mean],'rms_var':[rms_var],'spectral_centroid_mean':[spectral_centroid_mean],
                             'spectral_centroid_var':[spectral_centroid_var],'spectral_bandwidth_mean':[spectral_bandwith_mean],'spectral_bandwidth_var':[spectral_bandwith_var],
                             'rolloff_mean':[rolloff_mean],'rolloff_var':[rolloff_var],'zero_crossing_rate_mean':[zero_crossing_rate_mean],'zero_crossing_rate_var':[zero_crossing_rate_var],
                             'harmony_mean':[harmony_mean],'harmony_var':[harmony_var],'tempo':[tempo],})
    

    features = features.reindex(columns=cols_when_model_builds)

    # features = features.reshape(-1,1)
    return features



# Main

* 'blues':0, 'classical':1, 'country':2, 'disco':3, 'hiphop':4 ,'jazz':5 ,'metal':6, 'pop':7,'reggae':8 ,'rock':9 

In [42]:
features1 =  extract_features(r"..\music\Beyoncé-Single Ladies.mp3")
# features2 =  extract_features(r"..\music\Outkast-Hey Ya!.mp3")
# features3 =  extract_features(r"..\music\Beyoncé-Single Ladies.mp3")

# features3=  extract_features(r"..\data\Data\genres_original\pop\pop.00005.wav")



chroma_sft_mean:  0.44487435
chroma_sft_var:  0.09353361
rms_mean:  0.16312669
rms_var:  0.0012979151
spectral_centroid_mean:  2935.3469665270536
spectral_centroid_var:  398891.0507736011
spectral_bandwith_mean:  2719.901944654587
spectral_bandwith_var:  83637.97054642365
rolloff_mean:  6040.71543340452
rolloff_var:  1651437.4784093106
zero_crossing_rate_mean:  0.13704272207504326
zero_crossing_rate_var:  0.002931172480525642
harmony_mean:  -2.2207041e-06
harmony_var:  0.011054042
tempo:  95.703125


In [80]:
def confidence_score(proba):
    from collections import Counter
    confi = {}
    i=0
    for val in proba[0]:
        rounded = round(val *100,2)
        # print(rounded ,"% :",target_name[i])
        confi[target_name[i]] = rounded
        i = i+1

    k = Counter(confi)
    
    # Finding 3 highest values
    high = k.most_common(3) 
    
    for i in high:
        print(i[0]," :",i[1]," ")

In [83]:
features1 =  extract_features(r"..\music\Outkast-Hey Ya!.mp3")

print('---------------------------------------------------')

genre1 = xgb.predict(features1)
genreP = xgb.predict_proba(features1)
# genre2 = xgb.predict(features2)
# genre2 = xgb.predict_proba(features2)

# genre3 = xgb.predict(features3)

features1['label'] = genre1[0]
# features2['label'] = genre2[0]
# features3['label'] = genre3[0]


confidence_score(genreP)

# label1 = label_encoder.inverse_transform(genre1)[0]

# print(label1)
# print(label_encoder.inverse_transform(genre2))
# print(label_encoder.inverse_transform(genre3))


chroma_sft_mean:  0.48688152
chroma_sft_var:  0.07453386
rms_mean:  0.18483745
rms_var:  0.0010621852
spectral_centroid_mean:  2057.5505880081246
spectral_centroid_var:  138830.8042402148
spectral_bandwith_mean:  2235.3802284197614
spectral_bandwith_var:  52725.014757933415
rolloff_mean:  4201.572520697295
rolloff_var:  760465.70219222
zero_crossing_rate_mean:  0.09362877992848259
zero_crossing_rate_var:  0.000869912190093979
harmony_mean:  6.2925446e-06
harmony_var:  0.015244796
tempo:  107.666015625
---------------------------------------------------
metal  : 80.52  
rock  : 12.76  
disco  : 2.65  


In [44]:
print("Similiar Songs:",find_sim(features1))
print('---------------------------------------------------')

new_features = features1
new_features['tempo'] = 400
print(find_sim(new_features))
print('---------------------------------------------------')

combined_df = pd.concat([features1, features2], ignore_index=True)
combined_df = pd.concat([combined_df, features3], ignore_index=True)
combined_df = combined_df.drop(['filename'], axis=1)


Similiar Songs: filename
rock.00040.wav    0.894860
rock.00047.wav    0.857442
rock.00043.wav    0.857361
Name: test, dtype: float64
---------------------------------------------------
filename
hiphop.00015.wav    0.869704
rock.00022.wav      0.804720
jazz.00077.wav      0.799673
Name: test, dtype: float64
---------------------------------------------------


In [45]:
feature = 'tempo'


pred = find_pred(combined_df, features1, feature)


print("Predicted feature: ",pred)

pred_features = features1
pred_features[feature] = pred

print("Predicted Song: " ,find_sim(pred_features))

Predicted feature:  95.70312505049517
Predicted Song:  filename
rock.00040.wav    0.894860
rock.00047.wav    0.857442
rock.00043.wav    0.857361
Name: test, dtype: float64


In [46]:
# %pip install tensorflow